## Connecting to little_lemon_db database

In [15]:
import pandas as pd
import mysql.connector
import warnings

# Filter out pandas UserWarning
warnings.filterwarnings("ignore", category=UserWarning)


# Modify these parameters according to your MySQL configuration
mysql_config = {
    'host': 'localhost',
    'database': 'little_lemon_db',
    'user': 'awf',
    'password': 'awf'
}

# Establish connection
connection = mysql.connector.connect(**mysql_config)


# Query to fetch table names
query = "SELECT table_name FROM information_schema.tables WHERE table_schema = %s"
table_schema = (mysql_config['database'],)  # Using the configured database name
cursor = connection.cursor()
cursor.execute(query, table_schema)

# Fetch all table names
table_names = cursor.fetchall()

# List all for easier viewing
for tb in table_names:
    print(tb)

# Close connection
#connection.close()

('Bookings',)
('Courses',)
('Customers',)
('Food_Orders_Delevery_Status',)
('Starters',)


In [9]:
# Query the 'Customers' Table

query = "SELECT * FROM Customers"
df_customers = pd.read_sql(query, connection)
#df_orders.head(3)
df_customers

,CustomerID,FullName,PhoneNumber
0,1,Vanessa McCarthy,757536378
1,2,Marcos Romero,757536379
2,3,Hiroki Yamane,757536376
3,4,Anna Iversen,757536375
4,5,Diana Pinto,757536374


In [10]:
# Query the 'Bookings' Table

query = "SELECT * FROM Bookings"
df_bookings = pd.read_sql(query, connection)
df_bookings

,BookingID,BookingDate,TableNumber,NumberOfGuests,CustomerID
0,10,2021-11-11,7,5,1
1,11,2021-11-10,5,2,2
2,12,2021-11-10,3,2,4


## 1 - MySQL Aliases

**Problem Statement**:
Some of the tables and columns in the database are too long and difficult to read. Which is causes issues with the ouput of query. 
There need to find a way to generate results that are simpler to use, read, and understand.

To make the query easier to read and write, aliases are used to create a short name for a table or column.

**Objective**:
- Understand the concept of an alias
- Indetify situations in which you can use an alias
- Demonstrate the use of an alias in a query
- Use an alias to create a short name for a table or column


Aliases are used to create a short name for a table or column. It makes the query easier to read and write. The following aliases are used in the query:

- `c` is the alias for the `customers` table
- `o` is the alias for the `orders` table
- `p` is the alias for the `products` table
- `od` is the alias for the `orderdetails` table
  
Little Lemon is a company that sells products to customers. The company has a database called `little_lemon_db` . 


Alias can be used with CONCAT() function to combine two or more columns, and to give the combined columns a name.


Let's review each senario with an example.

1. Renaming a table using an alias
```sql
SELECT * FROM customers AS c;


# SQL alias syntax: renaming tables and columns
SELECT column1_name AS column1_alias, column2, column3
FROM table_name;

# multiple aliases
SELECT column1_name AS column1_alias, column2, column3, column4_name AS column4_alias
FROM table_name;


# Example
SELECT ClientOrderInformation as Orders
FROM ClientOrders;
```

2. Concatenating functions that combine output from two or more columns
into a single column
```sql
# SQL alias syntax: concatenating functions
SELECT CONCAT(column1, " ", column2) AS new_column_name
FROM table_name;

# Example
SELECT CONCAT(first_name, " ", last_name) AS `client_name`
FROM client_details;
```

3. Querying Multiple Tables

- The first thing to not when querying multiple tables is that you can use a one character alias to represent each table in the query. 

- For example if you are querying two different tables Table_1 and Table_2, you can use the alias t1 for Table_1 and t2 for Table_2.


First thing to do is to create an alias for each table in the query. Then, use the alias to specify the columns you want to retrieve from each table. 

```sql
# SQL alias syntax: querying multiple tables
SELECT t1.column1_name, t2.column2_name
FROM table1 t1, table2 t2
WHERE t1.column1_name = t2.column2_name;

# or 
SELECT x.column1, x.column2, y.column1, y.column2
FROM table1 AS x, table2 AS y
WHERE x.column1 < 12 AND y.column2 < 5;
```


### 1.1  - Little Lemon and MySQL aliases

Use your knowledge of aliases to help Little Lemon generate results that are simpler to use, read, and understand.

Renaming a table using an alias
```sql
SELECT 
    OrderID, 
    Date_food_order_placed_with_supplier as "Order Placed Date",
    Date_food_order_received_from_supplier as "Order Received Date",
    OrderStatus as "Order Status"
FROM Food_Orders_Delevery_Status;
```

In [16]:
query = """
SELECT 
    OrderID, 
    Date_food_order_placed_with_supplier as "Order Placed Date",
    Date_food_order_received_from_supplier as "Order Received Date",
    OrderStatus as "Order Status"
FROM Food_Orders_Delevery_Status;
"""

result = pd.read_sql(query, connection)
result

,OrderID,Order Placed Date,Order Received Date,Order Status
0,1,2022-04-05,2022-04-05,Delivered
1,2,2022-03-08,2022-10-08,Delivered
2,3,2022-05-19,None,In Progress
3,4,2022-01-10,2022-01-15,Delivered


Concatenating functions that combine output from two or more columns into a single column
```sql
# Concatenate OrderID and OrderStatus
SELECT 
    CONCAT(OrderID, " ", OrderStatus) AS "Orde_ID_Status"
FROM Food_Orders_Delevery_Status;
```

In [18]:
query = """
SELECT 
    CONCAT(OrderID, "-", OrderStatus) AS "Orde_ID_Status"
FROM Food_Orders_Delevery_Status;
"""

result = pd.read_sql(query, connection)
result

,Orde_ID_Status
0,1-Delivered
1,2-Delivered
2,3-In Progress
3,4-Delivered


In [20]:
query = "SELECT * FROM Starters"
df_Starters = pd.read_sql(query, connection)
df_Starters

,StarterName,Cost
0,Garlic Bread,5.0
1,Bruschetta,6.0
2,Calamari,8.0
3,Caprese Salad,7.0
4,Mozzarella Sticks,6.0


In [23]:
query = "SELECT * FROM Courses"
df_Courses = pd.read_sql(query, connection)
df_Courses

,CoursesName,Cost
0,Spaghetti Carbonara,12.0
1,Lasagna,14.0
2,Fettuccine Alfredo,13.0
3,Penne alla Vodka,15.0
4,Ravioli,16.0


**Multiple tables in the database**

The restaurant as divided their Menu into two tables call `Starters` and `Courses`.  Both tables shows the menus that are available to order and their respectif cost.

As part of their new promotional campaign, little lemon wants to promote starters that courses 6.00 or less and courses that cost 12.00 or less. 

So you need to query theses tables and indentify the meals that meet the criteria.

```sql
SELECT 
    s.StarterName,
    s.Cost,
    c.CourseName,
    c.Cost
FROM Starters AS s, Courses AS c
WHERE s.Cost <= 6.00 AND c.Cost <= 12.00;
``` 

In [25]:
query = """
SELECT 
    s.StarterName,
    s.Cost,
    c.CoursesName,
    c.Cost
FROM Starters AS s, Courses AS c
WHERE s.Cost <= 7.00 AND c.Cost <= 12.00;

"""
result = pd.read_sql(query, connection)
result

,StarterName,Cost,CoursesName,Cost
0,Mozzarella Sticks,6.0,Spaghetti Carbonara,12.0
1,Caprese Salad,7.0,Spaghetti Carbonara,12.0
2,Bruschetta,6.0,Spaghetti Carbonara,12.0
3,Garlic Bread,5.0,Spaghetti Carbonara,12.0


## 2 - Joining Tables


**Introduction**:
Lucky_Shrub need to gather information on their customers and the orders they have placed but the records are held in three different tables. Howerver, they can extract this information from their database using join clauses the requires table together.

**Objective**:
In this section, we will discover how the join clause works and how to use it to combine data from two or more tables.

- Understand the JOIN concept
- Demonstrate the use of the JOIN clause to combine data from two or more tables
- Describe the main types of JOIN clauses


**Problem Statement**:
Lucky_Shrub need to determine what products where ordered and which customers placed the orders. However, the information is stored in three different tables `Customers`, `Orders`, and `Products`.

So how can Lucky_Shrub extract records from three different tables?

The answer is to use the JOIN clause to combine data from two or more tables.

**The SQL JOIN clause -  Squerry data base on common column**:
The JOIN clause is used to combine rows from two or more tables, based on a related column between target tables.

For example :
- `CustomerID` : is a common column between the `Customers` and `Orders` tables
- `ProductID` : is a common column between the `Orders` and `Products` tables

These common columns are used to combine the data from the tables together and extract the records that are needed.

**Types of JOIN clauses**:
- `INNER JOIN`: returns records that have matching values in both tables
- `LEFT JOIN` : returns all records from the left table, and the matched records from the right table
- `RIGHT JOIN` : returns all records from the right table, and the matched records from the left table
- `FULL JOIN` : returns all records when there is a match in either left or right table
- `SELF JOIN` : is used to join a table to itself as if the table were two tables, temporarily renaming at least one table in the SQL statement







###  2.1 Inner JOIN

The `INNER JOIN` keyword selects records that have matching values in both tables.

```sql
# Inner Join Syntax
SELECT table1_name.column_name
FROM table1_name
INNER JOIN table2_name
ON table1_name.column_name = table2_name.column_name;



# or
SELECT table1.column1, table2.column2...
FROM table1
INNER JOIN table2
ON table1.column_name = table2.column_name;

# Example
SELECT 
    c.CustomerID,
    c.CustomerName,
    o.OrderID,
    o.OrderDate
FROM Customers AS c
INNER JOIN Orders AS o
ON c.CustomerID = o.CustomerID;
```



### 2.2 Left JOIN

The `LEFT JOIN` keyword returns all records from the left table (table1), and the matched records from the right table (table2). The result is NULL from the right side if there is no match.

```sql
# Left Join Syntax
SELECT table1_name.column_name
FROM table1_name
LEFT JOIN table2_name
ON table1_name.column_name = table2_name.column_name;


# example
SELECT 
    c.CustomerID,
    c.CustomerName,
    o.OrderID,
    o.OrderDate
FROM Customers AS c
LEFT JOIN Orders AS o
ON c.CustomerID = o.CustomerID;


# or 
SELECT 
    table1alias.column1name AS "columnn 1 new name", 
    table2alias.column2name AS "column 2 new name",
    table2alias.column1name AS "column 1 new name",
    table2alias.column2name 
FROM table1name AS table1alias
LEFT JOIN table2name AS table2alias
ON table1alias.column1name = table2alias.column1name;
```



### 2.3 Right JOIN

The `RIGHT JOIN` keyword returns all records from the right table (table2), and the matched records from the left table (table1). The result is NULL from the left side when there is no match.

It syntax if very similar to the `LEFT JOIN` clause.
The only difference is that the `RIGHT JOIN` clause returns all records from the right table and the matched records from the left table.

```sql
# Right Join Syntax
SELECT table1_name.column_name
FROM table1_name
RIGHT JOIN table2_name
ON table1_name.column_name = table2_name.column_name;

# or 
SELECT 
    table1alias.column1name AS "columnn 1 new name", 
    table2alias.column2name AS "column 2 new name",
    table2alias.column1name AS "column 1 new name",
    table2alias.column2name 
FROM table1name AS table1alias
RIGHT JOIN table2name AS table2alias
ON table1alias.column1name = table2alias.column1name;

# Example
SELECT 
    c.CustomerID,
    c.CustomerName,
    o.OrderID,
    o.OrderDate
FROM Customers AS c
RIGHT JOIN Orders AS o
ON c.CustomerID = o.CustomerID;
```


### Joins using little_lemon_join_db

In [26]:
import pandas as pd
import mysql.connector
import warnings

# Filter out pandas UserWarning
warnings.filterwarnings("ignore", category=UserWarning)


# Modify these parameters according to your MySQL configuration
mysql_config = {
    'host': 'localhost',
    'database': 'little_lemon_joins_db',
    'user': 'awf',
    'password': 'awf'
}

# Establish connection
connection = mysql.connector.connect(**mysql_config)


# Query to fetch table names
query = "SELECT table_name FROM information_schema.tables WHERE table_schema = %s"
table_schema = (mysql_config['database'],)  # Using the configured database name
cursor = connection.cursor()
cursor.execute(query, table_schema)

# Fetch all table names
table_names = cursor.fetchall()

# List all for easier viewing
for tb in table_names:
    print(tb)

('Bookings',)
('Customers',)


In [33]:
query = "SELECT * FROM Customers"
df_customers = pd.read_sql(query, connection)
#df_orders.head(3)
df_customers

,CustomerID,FullName,PhoneNumber
0,1,Vanessa McCarthy,757536378
1,2,Marcos Romero,757536379
2,3,Hiroki Yamane,757536376
3,4,Anna Iversen,757536375
4,5,Diana Pinto,757536374


In [34]:
query = "SELECT * FROM Bookings"
df_bookings = pd.read_sql(query, connection)
#df_orders.head(3)
df_bookings

,BookingID,BookingDate,TableNumber,NumberOfGuests,CustomerID
0,10,2021-11-11,7,5,1
1,11,2021-11-10,5,2,2
2,12,2021-11-10,3,2,4


**Task 1**: Little Lemon want a list of all customers who have made bookings. Write an INNER JOIN SQL statement to combine the full name and the phone number of each customer from the Customers table with the related booking date and 'number of guests' from the Bookings table. 
![Alt text](image-2.png)

```sql
SELECT c.FullName, c.PhoneNumber, b.BookingDate, b.NumberOfGuests
FROM Customers c
INNER JOIN Bookings b
ON c.CustomerID = b.CustomerID;

# or 
SELECT Customers.FullName, Customers.PhoneNumber, Bookings.BookingDate, Bookings.NumberOfGuests 
FROM Customers INNER JOIN Bookings 
ON Customers.CustomerID = Bookings.CustomerID;
```

In [35]:
query = """
SELECT 
    Customers.FullName, 
    Customers.PhoneNumber, 
    Bookings.BookingDate, 
    Bookings.NumberOfGuests 
FROM Customers 
INNER JOIN Bookings 
ON Customers.CustomerID = Bookings.CustomerID;
"""

result = pd.read_sql(query, connection)
result

,FullName,PhoneNumber,BookingDate,NumberOfGuests
0,Vanessa McCarthy,757536378,2021-11-11,5
1,Marcos Romero,757536379,2021-11-10,2
2,Anna Iversen,757536375,2021-11-10,2


In [37]:
query = """
SELECT 
    c.FullName, 
    c.PhoneNumber, 
    b.BookingDate, 
    b.NumberOfGuests
FROM Customers AS c
INNER JOIN Bookings AS b
ON c.CustomerID = b.CustomerID;
"""

result = pd.read_sql(query, connection)
result

,FullName,PhoneNumber,BookingDate,NumberOfGuests
0,Vanessa McCarthy,757536378,2021-11-11,5
1,Marcos Romero,757536379,2021-11-10,2
2,Anna Iversen,757536375,2021-11-10,2



**Task 2**: Little Lemon want to view information about all existing customers with bookings that have been made so far. This data must include customers who haven’t made any booking yet. 

![Alt text](image-3.png)


**Solution**:
```sql
SELECT c.CustomerID, b.BookingID
FROM Customers c
LEFT JOIN Bookings b
ON c.CustomerID = b.CustomerID;

# or
SELECT Customers.CustomerID, Bookings.BookingID 
FROM Customers LEFT JOIN Bookings 
ON Customers.CustomerID = Bookings.CustomerID;
```

In [36]:
query = """
SELECT c.CustomerID, b.BookingID
FROM Customers c
LEFT JOIN Bookings b
ON c.CustomerID = b.CustomerID;
"""

result = pd.read_sql(query, connection)
result

,CustomerID,BookingID
0,5,NaN
1,4,12.0
2,3,NaN
3,1,10.0
4,2,11.0


In [38]:
query = """
SELECT Customers.CustomerID, Bookings.BookingID 
FROM Customers LEFT JOIN Bookings 
ON Customers.CustomerID = Bookings.CustomerID;
"""

result = pd.read_sql(query, connection)
result

,CustomerID,BookingID
0,5,NaN
1,4,12.0
2,3,NaN
3,1,10.0
4,2,11.0


### Great Ressources : 

- https://www.mysqltutorial.org/mysql-basics/mysql-join/
- https://www.w3schools.com/sql/sql_join.asp